In [1]:
PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR/kmer-mlst

# Get MLST profiles

In [2]:
wget https://pubmlst.org/data/alleles/senterica/aroC.tfa

--2020-01-18 15:57:29--  https://pubmlst.org/data/alleles/senterica/aroC.tfa
Resolving pubmlst.org (pubmlst.org)... 129.67.24.31
Connecting to pubmlst.org (pubmlst.org)|129.67.24.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 511143 (499K)
Saving to: ‘aroC.tfa’

aroC.tfa            100%[===================>] 499.16K   856KB/s    in 0.6s    

2020-01-18 15:57:30 (856 KB/s) - ‘aroC.tfa’ saved [511143/511143]



In [7]:
du -sh aroC.tfa

500K	aroC.tfa


Pick first 10 alleles from this file.

In [12]:
head -n 100 aroC.tfa > aroC-10.fasta

Align everything with clustalomega.

In [58]:
conda run --name clustalo clustalo -i aroC-10.fasta -o aroC-10.aligned.fasta --force
conda run --name clustalo clustalo -i aroC-10.fasta --outfmt phylip -o aroC-10.aligned.phy --force

Also build phylogenetic tree.

In [61]:
conda run --name phyml phyml -i aroC-10.aligned.phy | head -n 3



. Command line: phyml -i aroC-10.aligned.phy 


In [63]:
cat aroC-10.aligned.phy_phyml_tree.txt

(aroC_9:0.00635288,aroC_3:0.00206180,(aroC_6:0.00208228,(aroC_2:0.00000001,(aroC_1:0.00866459,(aroC_10:0.00000001,(aroC_4:0.00000001,(aroC_5:0.00423962,(aroC_8:0.00417796,aroC_7:0.00000001)0.000000:0.00000001)0.962873:0.00207799)0.977224:0.00207798)0.791440:0.00209504)0.791428:0.00209499)0.879313:0.00209523)0.991691:0.00428820);


In [85]:
python -c '
from Bio import AlignIO
align = AlignIO.read("aroC-10.aligned.fasta", "fasta")
#print(align)
print(align[0].id)
'

aroC_1


In [67]:
python -c '
from Bio import AlignIO
align = AlignIO.read("aroC-10.aligned.fasta", "fasta")

def check_freq(str):
    freq = {}
    min=1000
    min_char = None
    for c in set(str):
       freq[c] = str.count(c)
       if freq[c] < min:
           min = freq[c]
           min_char = c
    return (freq, min_char)

snp_col_row = {}
for i in range(align.get_alignment_length()):
    col = align[:, i]
    (cf, min_char) = check_freq(col)
    
    if len(cf) > 1:
        positions = [pos for pos, char in enumerate(col) if char == min_char]
        print(f"index: {i}, align: {col}")
        #print(cf)
        #print(min_char)
        #print(positions)
        
        snp_col_row[i] = positions
'

index: 8, align: CTTTTTTTTT
index: 53, align: CCTCCCCCTC
index: 62, align: CTTTTTTTTT
index: 119, align: CCTCCCCCTC
index: 132, align: TCCCCCCCCC
index: 149, align: CCCCCTCCCC
index: 185, align: CCCTTCTTTC
index: 239, align: CCCTTCTTCT
index: 248, align: GGAGGAGGAG
index: 258, align: GGGGGGGGAG
index: 273, align: CCCCCCCTCC
index: 299, align: TTTTCTTTTT
index: 365, align: AAAATAAAAA
index: 410, align: CTCTCTCCTT
index: 452, align: GGGGGGGAGG
index: 464, align: CTTCCTCCCC


In [91]:
python -c '
from Bio import AlignIO
align = AlignIO.read("aroC-10.aligned.fasta", "fasta")

def check_freq(str):
    freq = {}
    min=1000
    min_char = None
    for c in set(str):
       freq[c] = str.count(c)
       if freq[c] < min:
           min = freq[c]
           min_char = c
    return (freq, min_char)

snp_col_row = {}
for i in range(align.get_alignment_length()):
    col = align[:, i]
    (cf, min_char) = check_freq(col)
    
    if len(cf) > 1:
        left_edge = max(0, i - 15)
        right_edge = min(align.get_alignment_length(), left_edge + 31)
        kmer_slice = align[:, left_edge:right_edge]
        #print(kmer_slice)
        
        kmer_table = {}
        for j in kmer_slice:
            kmer_string = str(j.seq)
            if not kmer_string in kmer_table:
                kmer_table[kmer_string] = [j.id]
            else:
                kmer_table[kmer_string].append(j.id)
        
        #print(kmer_table)
        print(f"Position {i}")
        for kmer in kmer_table:
            print(f"\t{kmer}")
            for id in kmer_table[kmer]:
                print(f"\t\t{id}")    
'

Position 8
	GTTTTTCGCCCGGGACACGCGGATTACACCT
		aroC_1
	GTTTTTCGTCCGGGACACGCGGATTACACCT
		aroC_2
		aroC_3
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_9
		aroC_10
Position 53
	GAAATACGGCCTGCGCGATTACCGCGGCGGT
		aroC_1
	GAAATACGGCCTGCGCGATTACCGTGGCGGT
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	GAAATACGGCCTGCGTGATTACCGTGGCGGT
		aroC_3
		aroC_9
Position 62
	CCTGCGCGATTACCGCGGCGGTGGACGTTCT
		aroC_1
	CCTGCGCGATTACCGTGGCGGTGGACGTTCT
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	CCTGCGTGATTACCGTGGCGGTGGACGTTCT
		aroC_3
		aroC_9
Position 119
	AGCGGCAGGGGCGATCGCCAAGAAATACTTG
		aroC_1
	AGCGGCAGGGGCGATCGCCAAGAAATACCTG
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	AGCGGCAGGGGCGATTGCCAAGAAATACCTG
		aroC_3
		aroC_9
Position 132
	ATCGCCAAGAAATACTTGGCGGAAAAGTTCG
		aroC_1
	ATCGCCAAGAAATACCTGGCGGAAAAGTTCG
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	ATTGCCAAGAAATACCTGGCGGAAAAGTTCG
		aroC_3
		aroC_9
Position 149
	GGCGGAAA